In [29]:
import os
import gc
import csv
import argparse
import pickle

import torch
from torch.utils.data import DataLoader
import pandas as pd
import numpy as np
from scipy.interpolate import interp1d

from dataset import ICUSepsisDataset
from model import SepsisPredictionModel

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

OPTIMIZERS = {
    'adam': torch.optim.Adam,
    'adadelta': torch.optim.Adadelta,
    'sgd': torch.optim.SGD,
    'adagrad': torch.optim.Adagrad
}

argparser = argparse.ArgumentParser(description='Train mask detection nerual network')
argparser.add_argument('--data-path', type=str, required=True, dest='data_path', default='/home/student/data/')
argparser.add_argument('--epochs', type=int, dest='epochs', default=100)
argparser.add_argument('--optimizer', type=str, dest='optimizer', choices=OPTIMIZERS.keys(), default='adam')
argparser.add_argument('--lr', type=float, dest='lr', default=0.005)
argparser.add_argument('--print-steps', type=int, dest='print_steps')

args = argparser.parse_args(['--data-path', '/Users/snirlugassy/Projects/sepsis_early_detection/data', '--epochs', '1'])

print('====== TRAIN =======')
print('optimizer:', args.optimizer)
print('epochs:', args.epochs)
print('l-rate:', args.lr)
print('device:', device)
print('====================')

model = SepsisPredictionModel(input_size=len(ICUSepsisDataset.features))
model.to(device)

icu_train = ICUSepsisDataset(os.path.join(args.data_path, 'train'))
train_loader = DataLoader(icu_train, batch_size=1, shuffle=True)

loss = torch.nn.CrossEntropyLoss()
optimizer = OPTIMIZERS[args.optimizer](model.parameters(), lr=args.lr)
 

====== TRAIN =======
optimizer: adam
epochs: 1
l-rate: 0.005
device: cpu
Found 20000 files in /Users/snirlugassy/Projects/sepsis_early_detection/data/train


In [32]:
x,y = icu_train[0]
print(x.shape, y.shape, len(y))
output = model(x)
print(output.shape)

torch.Size([23, 9]) torch.Size([23]) 23
torch.Size([23, 2])


In [31]:
loss(output, y)

ValueError: Using a target size (torch.Size([23])) that is different to the input size (torch.Size([23, 2])) is deprecated. Please ensure they have the same size.

In [33]:
for x,y in icu_train:
    # ignore invalid samples
    if x is None or y is None:
        continue

    optimizer.zero_grad()

    x = x.to(device).unsqueeze(0)
    y = y.to(device)  # squeeze since batch_size=1
    N = len(y)

    # Forward pass
    output = model(x).view(N,2) # squeeze since batch_size=1
    L = loss(output, y)
    break

ValueError: Using a target size (torch.Size([23])) that is different to the input size (torch.Size([23, 2])) is deprecated. Please ensure they have the same size.

In [28]:
x.shape, y.shape, output.shape

(torch.Size([1, 23, 9]), torch.Size([1, 23]), torch.Size([1, 23, 2]))